In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/MyDrive/ADL/HW7/names-1.txt'

In [3]:
import torch
import torch.nn as nn
from collections import defaultdict
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import tqdm
import math
from collections import defaultdict

In [4]:
FILL_IN = "FILL_IN"

In [5]:
# Dictionaries, {idx -> ch} and {ch -> idx}
itos = defaultdict(int)
stoi = defaultdict(int)
# Embedding dimension, per character
d_model = 10
# Hidden dimension for RNN and also MLP Language Models
d_h = 200

# START = STOP token
stoi['.'] = 0
itos[0] = '.'

# Fill in stoi and itoi; loop over names.txt
idx = 1
for name in open(file_path, 'r'):
    name = name.lower().strip()
    for ch in name:
        if ch not in stoi:
            stoi[ch] = idx
            itos[idx] = ch
            idx += 1

In [6]:
assert len(stoi) == len(itos)
vocab_size = len(stoi)
assert vocab_size == 27

In [7]:
stoi

defaultdict(int,
            {'.': 0,
             'e': 1,
             'm': 2,
             'a': 3,
             'o': 4,
             'l': 5,
             'i': 6,
             'v': 7,
             's': 8,
             'b': 9,
             'p': 10,
             'h': 11,
             'c': 12,
             'r': 13,
             't': 14,
             'y': 15,
             'n': 16,
             'g': 17,
             'z': 18,
             'f': 19,
             'd': 20,
             'u': 21,
             'k': 22,
             'w': 23,
             'q': 24,
             'x': 25,
             'j': 26})

## Attention RNN Language Model
- For each name, run an RNN character by character
- Use the recursion x = Tanh()(Wh @ h + Wx @ x + bh + bx) and y = Softmax()(Wy h + by)
- Do not use the RNN Cell from PyTorch, do this manually as hinted below
- Use attention as below to get the pprediction at each time step and define the next hidden state
- Token embedding size is d_model; d_h is the hidden size

In [8]:
class AttentionRNNLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # For each token, we have an embedding of size d_model
        self.e = nn.Embedding(vocab_size, d_model)

        # Wh and Wx act on h_t and x_t, respectively
        self.Wh = nn.Linear(d_h, d_h)
        self.Wx = nn.Linear(d_model, d_h)

        # We will concatenate h_t and k_t to predict the y_t vector
        self.Wy = nn.Linear(2 * d_h, vocab_size)

        # Attention is used to get scores e_{t, s} = h_s @ Wa @ h_t
        self.Wa = nn.Linear(d_h, d_h, bias=False)

    """
    Example schematic of this model:
       c1    c2    c3    c4    c5

       ^     ^     ^     ^     ^
       |     |     |     |     |

h0 ->  h1 -> h2 -> h3 -> h3 -> h4

       ^     ^     ^     ^     ^
       |     |     |     |     |

       c0    c1    c2    c3    c4

    As an example, to predict c3 take h3 with h2, h1
    a3i = h3 @ hi with i <= 3
    Apply softmax to a3i
    k3 = a33 @ h3 + a32 @ h2 + a31@h1
    Use k3 and h3 to predict c3
    k3 really is [k3, h3] first and then we get the logits for c3

    """

    # h_past here is the past history of hidden states
    # At time 1, it is just h0
    # At time 2, it is [h0, h1]
    def forward(self, x, h_past):
        # Run through to get the embedding for the token
        # The embedding per token is the feature vector x  we pass into the
        x = self.e(x)

        # Get the last hidden state
        h = h_past[-1]

        # Use the last hidden state and the current token to get the new hidden state
        # Pass through Tanh the combination of x and the last hidden state
        h = torch.tanh(self.Wh(h) + self.Wx(x))

        # Append the current state to the old states
        # (h0, ..., h_{t-1}) becomes (h0, ..., h_{t-1}, h_t)
        h_past = h_past + [h]

        # Get the scores, which are given by e_{t, s} = h_s @ Wa @ h_t, a scalar
        # Get all the real valued scores [e_{t, 0}, ..., e_{t, t}]
        e = torch.cat([torch.matmul(h_s, self.Wa(h).T) for h_s in h_past], dim=0)

        # From the scores, get the probabilities
        # a_{t, 0} + ... + a_{t, t} = 1
        a = torch.softmax(e, dim=0)

        # Define k_t as a_{t, 0} * h_0 + ... + a_{t, t} * h_t
        k = sum(a[i] * h_past[i] for i in range(len(h_past)))

        # Reset and define k_t as (k_t, h_t)
        k = torch.cat((k, h), dim=-1)

        # Get the logits which are Wy(k_t)
        z = self.Wy(k)

        # Return the logits and the current history of hidden states
        return z, h_past

In [9]:
model = AttentionRNNLanguageModel()
# Try some learning rates of the type 10^{x} where x is between -1 and -10
# Things should be going down, as below; use Adam
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [10]:
total_loss = 0
total_ct = 0
total_epochs = 10

for _ in range(total_epochs):
    for name in open(file_path, 'r'):
        # Lowercase and get rid of new lines and spaces at the end
        name = name.lower().strip()
        # Add the START and END padding token
        name = '.' + name + '.'

        # Get x_data, which is names[:-1]
        # Get y_data, which is names[1:]
        x_data = torch.tensor([stoi[ch] for ch in name[:-1]])
        y_data = torch.tensor([stoi[ch] for ch in name[1:]])

        # Set the list of all logits
        logits = []

        # Reset gradients
        optimizer.zero_grad()

        # Set the hidden state to random
        # (1, d_h)
        h_past = [torch.randn(1, d_h)]

        # Go through each token of a word and get the logits
        for i, x in enumerate(x_data):
            z, h_past = model(torch.tensor([x]), h_past)
            logits.append(z)

        # Concatenate all the logits
        logits = torch.cat(logits, dim=0)

        # Compute the loss
        loss = F.cross_entropy(logits, torch.tensor(y_data))

        # Get the new gradient
        loss.backward()

        # Clip the gradients at max norm 0.1
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)

        # Do a gradient update
        optimizer.step()

        # Get the loss for the batch and get the number of batches
        total_loss += loss.item()
        total_ct += 1

        if total_ct and total_ct % 100 == 0:
            print(total_loss / total_ct)
            total_loss = 0
            total_ct = 0

<ipython-input-10-dc092279c12d>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = F.cross_entropy(logits, torch.tensor(y_data))


3.219913079738617
2.955585696697235
2.6829817485809326
2.6000788962841033
2.545857068300247
2.544486765861511
2.3694819951057435
2.3475799560546875
2.4050630104541777
2.454604675769806
2.428211797475815
2.3703474307060244
2.380388709306717
2.4289652597904205
2.3814727485179903
2.3295285403728485
2.37736829161644
2.364897071123123
2.4031360602378844
2.353936940431595
2.432830696105957
2.359231711626053
2.3484823739528657
2.4070877265930175
2.3721742022037504
2.3397230136394502
2.3181908559799194
2.3396612572669984
2.3240962719917295
2.323481446504593
2.304518301486969
2.3679187285900114
2.314567801952362
2.3278211784362792
2.3223706781864166
2.2743859493732455
2.370546623468399
2.390475569963455
2.296295313835144
2.2917509615421294
2.3229856312274935
2.3899784016609193
2.2727492237091065
2.3425348234176635
2.283402738571167
2.2910234236717226
2.298567110300064
2.2595776820182802
2.262826752662659
2.3014712488651274
2.2980125164985656
2.3473772978782654
2.3101744174957277
2.3374480521678

In [11]:
with torch.no_grad():
    # Get perplexity
    sumneglogp = 0
    T = 0
    for name in open(file_path, 'r'):
        name = name.lower().strip()
        T += (len(name) + 1)
        # Pad with START and STOP
        name = '.' + name + '.'
        # Get the name from index 0 to -1 exclusive end
        x_data = [stoi[ch] for ch in name[:-1]]
        # Get the y from index 1 to end inclusive end
        y_data = [stoi[ch] for ch in name[1:]]

        # Define the logits per token prediction
        logits = []

        # Initialize the h vector to random
        h_past = [torch.randn(1, d_h)]

        # Loop over each chracter in the name and pass h and this into the RNN
        # Get the new logit
        for x in x_data:
            # x_data above was raw; get the int representation if you also do that
            x = torch.tensor(x)
            z, h_past = model(x, h_past)
            # Append the logit
            logits.append(z)

        logits = torch.cat(logits, dim=0)

        # Compute the loss
        loss = F.cross_entropy(logits, torch.tensor(y_data), reduction='sum')

        # Change to log base 2
        # log2(x) = ln(x) / ln(2)
        loss *= math.log2(math.e)

        sumneglogp += loss.item()

    # sumneglogp is -log(p('.' + name1)) -log(p('.' + name2)) -log(p('.' + name3)) ...
    # Divide by the appropriate term to get the answer we want
    print('Perplexity: ', 2 ** (sumneglogp / T))

Perplexity:  9.153936056617807


In [17]:
# Generate a random word using this distributon
# Intialize the word with
name = '.'
# Set this to [h_0]
h_past = [torch.randn(1, d_h)]
while True:
    # Get the last character in the name
    c = name[-1]
    # Make the distribution from c to any other word other than START
    logits, h_past = model(torch.tensor([stoi[c]]), h_past)
    # Get the probabilities
    p = torch.softmax(logits, dim=0)
    # Sample a character from th distribution above
    c = torch.multinomial(p, num_samples=1)

    # Halt generation if the token gotten is '.' or append the new token to the name
    if itos[c.item()] == '.':
        break
    else:
        name += itos[c.item()]

print('Generated name: ' , name[1:])

Generated name:  mkutib


## Gated CNN
- Implement a language model across names similar to that here: https://arxiv.org/abs/1612.08083v3
- Batch size will be 1
- For a name like "abcdef" we will use data [.abcdef] -> [abcdef.]
- '.' is a start and end token, which denotes either the start or end of a word

In [18]:
class GatedCNNLanguageModel(nn.Module):
    def __init__(self, n_layers, kernel_size):
        super().__init__()

        # Define an embedding layer so each character gets an embedding of size d_model
        # There are vocab_size tokens
        self.e = nn.Embedding(vocab_size, d_model)

        # Define n_layers
        self.n_layers = n_layers

        # Define A as the first Conv1d layer which takes in d_model returns d_model and uses kernel_size kernels
        self.A = nn.Conv1d(d_model, d_model, kernel_size, padding=0)

        # Define B as the first Conv1d layer which takes in d_model returns d_model and uses kernel_size kernels
        self.B = nn.Conv1d(d_model, d_model, kernel_size, padding=0)

        # Define a list of A layers as above; n_layers
        self.A_list = nn.ModuleList([nn.Conv1d(d_model, d_model, kernel_size, padding=0) for _ in range(n_layers)])

        # Define a list of B layers as above; n_layers
        self.B_list = nn.ModuleList([nn.Conv1d(d_model, d_model, kernel_size, padding=0) for _ in range(n_layers)])

        # Define a fully connected layer which takes d_model embedding and gives the logits across vocab_size tokens
        self.fc = nn.Linear(d_model, vocab_size)

        # Pad on the left with an amount such that the input is the same dimension as the output
        # Remember, our data looks like [a, b, c] -> [b, c, d]
        # We want to make sure tokens to the left of b (a) are used to predict b
        # We want to make sure tokens to the left of c (a, b) are used to predict c
        # We want to make sure tokens to the left of d (a, b, c) are used to predict d
        # Etc.
        # Pad with on the left
        self.pad = nn.ConstantPad1d((kernel_size - 1, 0), 0)

    def forward(self, x):
        # Run through to get the embedding for the token
        # The embedding per token is the feature vector x  we pass into the
        # (B, L, D)
        x = self.e(x)

        # Change the above to be (B, D, L)
        x = x.transpose(1, 2)

        # Pad on the left with the right amount
        # Remember, what goes in, need to come out the same dimension and you can't "look ahead"
        # You want to just pad on the left
        # (B, D, L + kernel_size - 1)
        x = self.pad(x)

        # Run x through A, B and get h as defined for the gated CNN
        # Define the res_input as the hidden layer we get; we'll move forward with h
        h = self.A(x) * torch.sigmoid(self.B(x))
        res_input = h

        for i, (conv, conv_gate) in enumerate(zip(self.A_list, self.B_list)):
            # Pad on the left
            h = self.pad(h)
            # Apply conv and conv_gate to h and then combine to get the new h
            h = conv(h) * torch.sigmoid(conv_gate(h))
            # Get the new h, and also add to it the old non transformed hidden layer; the residual connection
            # Reset the residual so you use it next time
            h = h + res_input
            res_input = h

        # Get the final h to be of dimension (B, L, D)
        h = h.transpose(1, 2)

        # Get the logits which is of dimension (B, L, vocab_size)
        logit = self.fc(h)

        # Transpose the logits to be of dimension (B, vocab_size, L)
        logit = logit.transpose(1, 2)

        return logit

In [19]:
n_layers = 3
kernel_size = 3
model = GatedCNNLanguageModel(n_layers, kernel_size)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [20]:
total_loss = 0
total_ct = 0
total_epochs = 10

for _ in range(total_epochs):
    for name in open(file_path, 'r'):
        name = name.lower().strip()
        # Pad the name with a special token for start and stop; use the same token
        name = '.' + name + '.'

        # Add the start and end padding token; reshape as needed
        # Batch size is 1 here
        x_data = [[stoi[ch] for ch in name[:-1]]]
        y_data = [[stoi[ch] for ch in name[1:]]]

        # Get the logits acrosss the x_data above
        logits = model(torch.tensor(x_data))

        # Compute the loss
        loss = F.cross_entropy(logits.reshape(-1, vocab_size), torch.tensor(y_data).view(-1))

        # Get the new gradient
        optimizer.zero_grad()
        loss.backward()

        # Clip the gradients at max norm 0.1
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)

        # Do a gradient update
        optimizer.step()

        # Get the loss for the batch and get the number of batches
        total_loss += loss.item()
        total_ct += 1

        if total_ct and total_ct % 100 == 0:
            print(total_loss / total_ct)
            total_loss = 0
            total_ct = 0

3.3451735591888427
3.3105281591415405
3.324157621860504
3.3050982928276063
3.3010795164108275
3.2847828221321107
3.2926138114929198
3.2811825680732727
3.2872261953353883
3.2782582902908324
3.2688193845748903
3.2499901819229127
3.270339994430542
3.2473795580863953
3.244728691577911
3.251039936542511
3.240408914089203
3.219811956882477
3.2321912598609925
3.214990983009338
3.2026428890228273
3.1826341557502746
3.1814533710479735
3.1955481481552126
3.167529170513153
3.1570703959465027
3.154725122451782
3.1389442777633665
3.1015731382369993
3.123248188495636
3.0947557306289672
3.106846318244934
3.086003615856171
3.0621798253059387
3.084063608646393
3.0517076230049134
3.0563987565040587
3.0695438265800474
3.0292426347732544
3.024872312545776
3.0505569410324096
2.9921898293495177
3.0013971495628358
3.0148068952560423
2.974077970981598
2.9741770219802857
2.996461715698242
2.9334070110321044
2.984472408294678
2.91647451877594
3.0302844142913816
2.913046565055847
2.9030767631530763
3.00664785623

In [21]:
with torch.no_grad():
    # Get perplexity
    sumneglogp = 0
    T = 0
    for name in open(file_path, 'r'):
        name = name.lower().strip()
        T += (len(name) + 1)
        name = '.' + name + '.'

        # Get the name from index 0 to -1 exclusive end
        x_data = [[stoi[ch] for ch in name[:-1]]]

        # Get the y from index 1 to end inclusive end
        y_data = [[stoi[ch] for ch in name[1:]]]

        # logits per token prediction
        logits = model(torch.tensor(x_data))

        # Compute the loss
        loss = F.cross_entropy(logits.reshape(-1, vocab_size), torch.tensor(y_data).view(-1), reduction='sum')

        # Change to log base 2
        # log2(x) = ln(x) / ln(2)
        loss *= 1 / math.log(2)

        # Update the sum across the negative logs
        sumneglogp += loss.item()

    # sumneglogp is -log(p('.' + name1)) -log(p('.' + name2)) -log(p('.' + name3)) ...

    # Divide by the appropriate term to get the answer we want
    print('Perplexity: ', 2 ** (sumneglogp / T))

Perplexity:  9.796306798491369


In [31]:
# Generate a random word using this distributon
# Intialize the word with
name = '.'
i_name = []
while True:
    # Get the last character and and add to i_name as an int representation
    i_name.append(stoi[name[-1]])

    # Make the distribution from c to any other word other than START
    logits = model(torch.tensor([i_name]))

    # Grab the last logits, for the new term
    # Alternatively, just push the appropriate amount into the model
    logits = logits[:, :, -1]

    # Get the probabilities
    p = torch.softmax(logits, dim=-1)

    # Sample from the above probability distribution
    c = torch.multinomial(p, num_samples=1)

    # If we sample 0, stop; otherwise, continue
    if c.item() == stoi['.']:
        break
    else:
        name += itos[c.item()]

print('Generated name: ' , name[1:])

Generated name:  omma
